In [10]:
import pandas as pd

In [11]:
df = pd.read_csv("lahore_housing_prices.csv")
df.head()

,house_id,Type,Location,Area,Bath(s),Bedroom(s),Price
0,46326643,House,"DHA Defence, Lahore, Punjab",1 Kanal,6,5,75500000
1,46952582,House,"Bahria Orchard, Lahore, Punjab",8 Marla,5,5,25000000
2,47357581,House,"Paragon City, Lahore, Punjab",10 Marla,6,5,47000000
3,47397157,Flat,"Askari, Lahore, Punjab",10 Marla,3,3,28000000
4,43109993,House,"DHA Defence, Lahore, Punjab",8 Marla,5,4,43500000


### Basic info related to the data

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17169 entries, 0 to 17168
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   house_id    17169 non-null  int64 
 1   Type        17169 non-null  object
 2   Location    17169 non-null  object
 3   Area        17169 non-null  object
 4   Bath(s)     17169 non-null  int64 
 5   Bedroom(s)  17169 non-null  int64 
 6   Price       17169 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 939.1+ KB


In [13]:
df.describe(include="all")

,house_id,Type,Location,Area,Bath(s),Bedroom(s),Price
count,1.716900e+04,17169,17169,17169,17169.000000,17169.000000,1.716900e+04
unique,NaN,4,324,170,NaN,NaN,NaN
top,NaN,House,"DHA Defence, Lahore, Punjab",5 Marla,NaN,NaN,NaN
freq,NaN,16017,4845,4304,NaN,NaN,NaN
mean,4.655362e+07,NaN,NaN,NaN,4.716233,4.183820,4.758766e+07
std,1.796472e+06,NaN,NaN,NaN,1.329716,1.294566,5.467632e+07
min,6.872350e+05,NaN,NaN,NaN,1.000000,1.000000,1.150000e+05
25%,4.652353e+07,NaN,NaN,NaN,4.000000,3.000000,1.800000e+07
50%,4.706818e+07,NaN,NaN,NaN,5.000000,4.000000,3.200000e+07
75%,4.731195e+07,NaN,NaN,NaN,6.000000,5.000000,6.000000e+07


In [14]:
df.isnull().sum()

house_id      0
Type          0
Location      0
Area          0
Bath(s)       0
Bedroom(s)    0
Price         0
dtype: int64

Since there are no Null values in any of the fields of our datasets, we don't need to do any data binning or dropping rows

In [15]:
def convert_to_marlas(area_str):
    area_str = str(area_str).lower()
    kanal = 0
    marla = 0

    if "kanal" in area_str:
        kanal_part = area_str.split("kanal")[0].strip()
        kanal = float(kanal_part) if kanal_part else 0  # Convert to float

    if "marla" in area_str:
        marla_part = area_str.split("marla")[0].split()[-1].strip()
        marla = float(marla_part) if marla_part else 0  # Convert to float

    return kanal * 20 + marla  # 1 Kanal = 20 Marlas

df['Area_Marlas'] = df['Area'].apply(convert_to_marlas)
df.head()

,house_id,Type,Location,Area,Bath(s),Bedroom(s),Price,Area_Marlas
0,46326643,House,"DHA Defence, Lahore, Punjab",1 Kanal,6,5,75500000,20.0
1,46952582,House,"Bahria Orchard, Lahore, Punjab",8 Marla,5,5,25000000,8.0
2,47357581,House,"Paragon City, Lahore, Punjab",10 Marla,6,5,47000000,10.0
3,47397157,Flat,"Askari, Lahore, Punjab",10 Marla,3,3,28000000,10.0
4,43109993,House,"DHA Defence, Lahore, Punjab",8 Marla,5,4,43500000,8.0


### Standardization
We need to stardard some of the fields so that there are no inconsistencies in the data.

In [16]:
df["Type"] = df["Type"].str.lower().str.strip()
df["Type"].value_counts()

Type
house        16017
flat          1146
penthouse        4
room             2
Name: count, dtype: int64

In [17]:
mask_1 = df['Type'] == 'room'
df = df[~mask_1]

mask_2 = df['Type'] == 'penthouse'
df = df[~mask_2]

since there are only **6** values for **2** categories in the **type** column, we drop them as they will affect the overall analysis 

In [18]:
df["Location"] = df["Location"].str.lower().str.strip()
df["Location"].value_counts()

Location
dha defence, lahore, punjab      4843
bahria town, lahore, punjab      1766
askari, lahore, punjab            852
raiwind road, lahore, punjab      599
johar town, lahore, punjab        511
                                 ... 
pakki thatti, lahore, punjab        1
lytton road, lahore, punjab         1
ali town, lahore, punjab            1
madina colony, lahore, punjab       1
rehman park, lahore, punjab         1
Name: count, Length: 324, dtype: int64

### Reducing Cardinality
there are 324 unique locations, which is too much for most models. so we group top **20** frequent locations and label the rest as **Others**. 

In [19]:
top_locations = df["Location"].value_counts().nlargest(20).index
df["Location"] = df["Location"].apply(lambda x: x if x in top_locations else "other")

### Converting Data types
Ensuring numeric columns are of numeric types

In [20]:
df["Bath(s)"] = pd.to_numeric(df["Bath(s)"], errors='coerce')
df["Bedroom(s)"] = pd.to_numeric(df["Bedroom(s)"], errors='coerce')
df["Price"] = pd.to_numeric(df["Price"], errors='coerce')
df["Area_Marlas"] = pd.to_numeric(df["Area_Marlas"], errors='coerce')

finally saving the preprocessed data into a new CSV file.

In [21]:
df.to_csv("preprocessed_lahore_house_prices.csv", index=False)